In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyodbc

In [2]:
# Setup lấy data
quiz_items=0
def scap (url):
    global quiz_items
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    response = requests.get(url, headers=headers)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    quiz_items = soup.find_all('ol', class_='wp-block-list')
    return quiz_items

print(scap('https://mcqprime.com/artificial-intelligence-mcq/')) # test

[<ol class="wp-block-list">
<li><strong>What does “AI bias” mean when used in relation to artificial intelligence?</strong><br/>a. The tendency of AI systems to make decisions without data<br/>b. The unintentional discrimination in AI systems due to biased training data<br/>c. The ability of AI systems to make decisions without human intervention<br/>d. The ethical considerations when designing AI algorithms</li>
</ol>, <ol class="wp-block-list" start="2">
<li><strong>Which AI technique is often used for clustering data into groups with similar characteristics?</strong><br/>a. Regression<br/>b. Reinforcement Learning<br/>c. Supervised Learning<br/>d. Unsupervised Learning</li>
</ol>, <ol class="wp-block-list" start="3">
<li><strong>What function does a Recurrent Neural Network (RNN) perform in AI?</strong><br/>a. To classify images<br/>b. To analyze text sentiment<br/>c. To process sequences of data with memory<br/>d. To play board games</li>
</ol>, <ol class="wp-block-list" start="4">

In [3]:
questions = []
optionA = []
optionB = []
optionC = []
optionD = []

In [4]:
# Tách câu hỏi và đáp án
def createdata(quiz_items):
    global questions
    global optionA 
    global optionB 
    global optionC 
    global optionD
    for item in quiz_items:
        # loại bỏ khoảng trắng
        text = item.text.strip()
        #text = text.replace('Xem đáp án', '') test web vietnam
        text = text.replace('"', '')
        
        # tách câu hỏi và đáp án
        parts = text.split("a.", 1)
        if len(parts) == 2:
            # câu hỏi
            question = parts[0].strip()
            question = ' '.join(question.split())  # xoá khoảng trắng và cách dòng 
            answers = parts[1]
            
            # đáp án
            options = answers.split("b.")
            if len(options) == 2:
                a = "A." + options[0].strip()
                rest = options[1]
                
                parts = rest.split("c.")
                if len(parts) == 2:
                    b = "B." + parts[0].strip()
                    rest = parts[1]
                    
                    parts = rest.split("d.")
                    if len(parts) == 2:
                        c = "C." + parts[0].strip()
                        d = "D." + parts[1].strip()
                        
                        questions.append(question)
                        optionA.append(a)
                        optionB.append(b)
                        optionC.append(c)
                        optionD.append(d)
    return questions, optionA, optionB, optionC, optionD

In [5]:
def createDf(quiz_items):
    global questions
    global optionA 
    global optionB 
    global optionC 
    global optionD
    createdata(quiz_items)
    df = pd.DataFrame({
        'Question': questions,
        'Option A': optionA, 
        'Option B': optionB,
        'Option C': optionC,
        'Option D': optionD
    })
    return df

In [9]:
# link 1 :  https://mcqprime.com/machine-learning-mcq/
# link 2 :  https://mcqprime.com/artificial-intelligence-mcq/
quiz_items=0
questions = []
optionA = []
optionB = []
optionC = []
optionD = []
quiz_items=scap('https://mcqprime.com/artificial-intelligence-mcq/')
df=createDf(quiz_items)
df.head(10)

,Question,Option A,Option B,Option C,Option D
0,What does “AI bias” mean when used in relation...,A.The tendency of AI systems to make decisions...,B.The unintentional discrimination in AI syste...,C.The ability of AI systems to make decisions ...,D.The ethical considerations when designing AI...
1,Which AI technique is often used for clusterin...,A.Regression,B.Reinforcement Learning,C.Supervised Learning,D.Unsupervised Learning
2,What function does a Recurrent Neural Network ...,A.To classify images,B.To analyze text sentiment,C.To process sequences of data with memory,D.To play board games
3,The main objective of artificial intelligence?,A.To replicate human intelligence exactly,B.To develop computer programs that can think ...,C.To mimic human behavior without understanding,D.To solve complex problems using algorithms
4,What does “NLU” mean when referring to natural...,A.Natural Language Understanding,B.Neural Language Unit,C.New Linguistic Understanding,D.Neutral Language Understanding
5,What is the name of the kind of AI system that...,A.Strong AI,B.Weak AI,C.Narrow AI,D.General AI
6,What is the purpose of an AI chatbot?,A.To play video games,B.To automate routine customer service tasks,C.To generate random text,D.To translate languages
7,Which AI application involves teaching a compu...,A.Supervised Learning,B.Unsupervised Learning,C.Reinforcement Learning,D.Machine Vision
8,What does “IoT” mean in terms of artificial in...,A.Internet of Things,B.Intelligence of Technology,C.Input of Text,D.Internet of Training
9,Which of the following is not a common applica...,A.Sentiment analysis,B.Machine translation,C.Image recognition,D.Chatbots


In [25]:
# save thành csv (test)
df.to_csv('quiz2.csv', index=True)

In [10]:

# tạo quiz1 trước sau đó tạo quiz2 ==> đổi tên quiz1 thành quiz2
# kết nối tới SQL Server
server = 'MINHLAP\SQLEXPRESS'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=master;Trusted_Connection=yes;')
cursor = conn.cursor()

# tạo database LAB1
cursor.execute('''
    IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'LAB1')
    BEGIN
        CREATE DATABASE LAB1
    END
    ''')
conn.commit()
conn.close()

# kết nối lab1
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=LAB1;Trusted_Connection=yes;')
cursor = conn.cursor()


# tạo table
cursor.execute('''
CREATE TABLE quiz2 (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    Question NVARCHAR(1000),
    OptionA NVARCHAR(500),
    OptionB NVARCHAR(500),
    OptionC NVARCHAR(500),
    OptionD NVARCHAR(500)
)
''')

# tạo list data từ dataframe == > nhập vào sql server bằng pyodbc (insert data)
data = [(row['Question'], row['Option A'], row['Option B'], row['Option C'], row['Option D']) 
        for index, row in df.iterrows()]

# thêm data vào table
cursor.executemany('''
INSERT INTO quiz2 (Question, OptionA, OptionB, OptionC, OptionD)
VALUES (?, ?, ?, ?, ?)
''', data)

conn.commit()
conn.close()
print("Data saved to SQL Server")

Data saved to SQL Server


<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\quang\AppData\Local\Temp\ipykernel_12700\4270974513.py:3: SyntaxWarning: invalid escape sequence '\S'
  server = 'MINHLAP\SQLEXPRESS'


In [4]:
# xem data đã lưu trong sql server
server = 'MINHLAP\SQLEXPRESS'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=LAB1;Trusted_Connection=yes;')
cursor = conn.cursor()

cursor.execute('SELECT * FROM quiz1')
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(rows, columns=columns)

df

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\quang\AppData\Local\Temp\ipykernel_17280\2188992686.py:2: SyntaxWarning: invalid escape sequence '\S'
  server = 'MINHLAP\SQLEXPRESS'


,ID,Question,OptionA,OptionB,OptionC,OptionD
0,1,What does “AI bias” mean when used in relation...,A.The tendency of AI systems to make decisions...,B.The unintentional discrimination in AI syste...,C.The ability of AI systems to make decisions ...,D.The ethical considerations when designing AI...
1,2,Which AI technique is often used for clusterin...,A.Regression,B.Reinforcement Learning,C.Supervised Learning,D.Unsupervised Learning
2,3,What function does a Recurrent Neural Network ...,A.To classify images,B.To analyze text sentiment,C.To process sequences of data with memory,D.To play board games
3,4,The main objective of artificial intelligence?,A.To replicate human intelligence exactly,B.To develop computer programs that can think ...,C.To mimic human behavior without understanding,D.To solve complex problems using algorithms
4,5,What does “NLU” mean when referring to natural...,A.Natural Language Understanding,B.Neural Language Unit,C.New Linguistic Understanding,D.Neutral Language Understanding
...,...,...,...,...,...,...
84,85,What is the name of the method used in AI that...,A.Reinforcement Learning,B.Supervised Learning,C.Unsupervised Learning,D.Semi-supervised Learning
85,86,What is the main objective of an AI system tha...,A.To summarize text documents,B.To convert speech to text,C.To translate text from one language to another,D.To generate human-like text
86,87,What is the name of a particular class of AI m...,A.Discriminative Model,B.Generative Model,C.Ensemble Model,D.Reinforcement Model
87,88,What does the term “hyperparameter” refer in AI?,A.Parameters learned by the model during training,B.Parameters that define the structure of the ...,C.Parameters used to make predictions,D.Parameters related to the loss function


In [3]:

server = 'MINHLAP\SQLEXPRESS'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=LAB1;Trusted_Connection=yes;')
cursor = conn.cursor()

cursor.execute('SELECT * FROM quiz2')
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(rows, columns=columns)

df


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\quang\AppData\Local\Temp\ipykernel_17280\422983743.py:1: SyntaxWarning: invalid escape sequence '\S'
  server = 'MINHLAP\SQLEXPRESS'


,ID,Question,OptionA,OptionB,OptionC,OptionD
0,1,Which of the following machine learning algori...,A.Decision Tree,B.Support Vector Machines (SVM),C.Random Forest,D.k-Nearest Neighbors (k-NN)
1,2,Identify the disadvantages of decision trees.,A.Prone to overfitting,B.Difficult to interpret,C.Require large amounts of training data,D.Slow to train
2,3,What is the term used to describe the process ...,A.Training,B.Learning,C.Modeling,D.Prediction
3,4,Machine learning is a subset of ______________...,A.Artificial intelligence (AI),B.Data science,C.Computer programming,D.Robotics
4,5,__________ is a machine learning technique whi...,A.Classification,B.Regression,C.Clustering,D.Anomaly detection
5,6,"Identify the type of learning, which is used f...",A.a. Supervised learning,B.Unsupervised learning,C.Reinforcement learning,D.Semi-supervised learning
6,7,Identify from the following in which dimension...,A.Feature extraction,B.Feature selection,C.Feature engineering,D.Feature augmentation
7,8,______________ is the father of machine learni...,A.Alan Turing,B.John McCarthy,C.Geoffrey Hinton,D.Andrew Ng
8,9,The most important phase in genetic algorithms...,A.Selection,B.Crossover,C.Mutation,D.Fitness evaluation
9,10,Which of the following issues in machine learn...,A.Classification,B.Regression,C.Clustering,D.All of the above


In [11]:
# Nối 2 bảng quiz1 và quiz2 thành 1 bảng quiz 
server = 'MINHLAP\SQLEXPRESS'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=LAB1;Trusted_Connection=yes;')
cursor = conn.cursor()

# tạo bảng QUIZ 
cursor.execute('''
CREATE TABLE QUIZ (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    Question NVARCHAR(1000),
    OptionA NVARCHAR(500),
    OptionB NVARCHAR(500),
    OptionC NVARCHAR(500),
    OptionD NVARCHAR(500)
)
''')

# thêm data từ quiz1 và quiz2 vào QUIZ
cursor.execute('''
INSERT INTO QUIZ (Question, OptionA, OptionB, OptionC, OptionD)
SELECT Question, OptionA, OptionB, OptionC, OptionD FROM quiz1
UNION ALL
SELECT Question, OptionA, OptionB, OptionC, OptionD FROM quiz2
''')

conn.commit()
conn.close()


<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\quang\AppData\Local\Temp\ipykernel_12700\721557669.py:2: SyntaxWarning: invalid escape sequence '\S'
  server = 'MINHLAP\SQLEXPRESS'


In [12]:
# check QUIZ

server = 'MINHLAP\SQLEXPRESS'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE=LAB1;Trusted_Connection=yes;')
cursor = conn.cursor()

cursor.execute('SELECT * FROM QUIZ')
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(rows, columns=columns)

df

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\quang\AppData\Local\Temp\ipykernel_12700\1004743098.py:3: SyntaxWarning: invalid escape sequence '\S'
  server = 'MINHLAP\SQLEXPRESS'


,ID,Question,OptionA,OptionB,OptionC,OptionD
0,1,Which of the following machine learning algori...,A.Decision Tree,B.Support Vector Machines (SVM),C.Random Forest,D.k-Nearest Neighbors (k-NN)
1,2,Identify the disadvantages of decision trees.,A.Prone to overfitting,B.Difficult to interpret,C.Require large amounts of training data,D.Slow to train
2,3,What is the term used to describe the process ...,A.Training,B.Learning,C.Modeling,D.Prediction
3,4,Machine learning is a subset of ______________...,A.Artificial intelligence (AI),B.Data science,C.Computer programming,D.Robotics
4,5,__________ is a machine learning technique whi...,A.Classification,B.Regression,C.Clustering,D.Anomaly detection
...,...,...,...,...,...,...
137,138,What is the name of the method used in AI that...,A.Reinforcement Learning,B.Supervised Learning,C.Unsupervised Learning,D.Semi-supervised Learning
138,139,What is the main objective of an AI system tha...,A.To summarize text documents,B.To convert speech to text,C.To translate text from one language to another,D.To generate human-like text
139,140,What is the name of a particular class of AI m...,A.Discriminative Model,B.Generative Model,C.Ensemble Model,D.Reinforcement Model
140,141,What does the term “hyperparameter” refer in AI?,A.Parameters learned by the model during training,B.Parameters that define the structure of the ...,C.Parameters used to make predictions,D.Parameters related to the loss function
